# Course Project
Author: Olga Chernytska
 
## Part 2. Modeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib

#### Step 0. Load data.

In [2]:
def load_data(direction):
    df = pd.read_csv('output/direction'+str(direction) + '_df_clean.csv')
    df = df[['drive_index','start_stop','end_stop','start_time','time_diff_seconds']]
    df['start_time'] = pd.to_datetime(df['start_time'], unit='ns')
    df['direction'] = direction
    
    return df

In [3]:
direction0_df = load_data(0)
direction1_df = load_data(1)

df_step0 = pd.concat([direction0_df, direction1_df])
df_step0 = df_step0.reset_index(drop = True)

In [4]:
print('Result: ')
print(df_step0.shape)
df_step0.head()

Result: 
(150864, 6)


,drive_index,start_stop,end_stop,start_time,time_diff_seconds,direction
0,8,stop #01: id=36853,stop #02: id=37283,2017-07-20 06:29:53.937044000,30.763969,0
1,8,stop #02: id=37283,stop #03: id=36822,2017-07-20 06:30:24.701013000,195.895895,0
2,8,stop #03: id=36822,stop #04: id=36823,2017-07-20 06:33:40.596908000,97.629325,0
3,8,stop #04: id=36823,stop #05: id=36821,2017-07-20 06:35:18.226233500,31.148851,0
4,8,stop #05: id=36821,stop #06: id=36812,2017-07-20 06:35:49.375084000,158.963154,0


#### Step 1. Add features.

In [5]:
def add_features(df):
    df['season'] = np.where(df['start_time'].dt.month.isin([12,1,2]), 'winter',
                             np.where(df['start_time'].dt.month.isin([3,4,5]), 'spring',
                             np.where(df['start_time'].dt.month.isin([6,7,8]), 'summer', 'autumn')))
    df['weekday'] = df['start_time'].dt.weekday_name
    df['hour'] = df['start_time'].dt.hour
    df['minute'] = (round(df['start_time'].dt.minute/15,0)*15 % 60).astype(int)
    df['route_interval'] = df['start_stop'].str.extract('(#\d*)', expand = False) + '_' \
                        + df['end_stop'].str.extract('(#\d*)', expand = False)
    return df

In [6]:
df_step1 = add_features(df_step0)

Rearrange to dummies:

In [7]:
def df_with_dummies(df):
    dummies_direction = pd.get_dummies(df['direction'], drop_first = True, prefix = 'is_direction')
    dummies_route_stops = pd.get_dummies(df['route_interval'], drop_first = True, prefix = 'is_stops')

    dummies_season = pd.get_dummies(df['season'], drop_first = True, prefix='is')
    dummies_weekday = pd.get_dummies(df['weekday'], drop_first = True, prefix = 'is')
    dummies_hour = pd.get_dummies(df['hour'], drop_first = True, prefix = 'is_hour')
    dummies_minute = pd.get_dummies(df['minute'], drop_first = True, prefix = 'is_minute')
    
    return df[['time_diff_seconds']].join([dummies_direction, dummies_route_stops,
                                      dummies_season, dummies_weekday, dummies_hour, dummies_minute] )

In [8]:
df_step1_dummies = df_with_dummies(df_step1)

In [9]:
print('Result: ')
print(df_step1_dummies.shape)
df_step1_dummies.head()

Result: 
(150864, 58)


,time_diff_seconds,is_direction_1,is_stops_#01_#02,is_stops_#02_#03,is_stops_#03_#04,is_stops_#04_#05,is_stops_#05_#06,is_stops_#06_#07,is_stops_#07_#08,is_stops_#08_#09,...,is_hour_13,is_hour_14,is_hour_15,is_hour_16,is_hour_17,is_hour_18,is_hour_19,is_minute_15,is_minute_30,is_minute_45
0,30.763969,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,195.895895,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,97.629325,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,31.148851,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,158.963154,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


#### Step 2. Build baseline model.

In [11]:
test, train = train_test_split(df_step1_dummies, test_size = 0.75)
X_test = test[test.columns[1:]]
y_test = test[test.columns[0]]

X_train = train[train.columns[1:]]
y_train = train[train.columns[0]]

In [12]:
regr = LinearRegression()
regr.fit(X_train, y_train)
scores = cross_val_score(regr, X_train, y_train, cv=5, scoring = 'neg_median_absolute_error')
print("Median absolute error: %0.2f (+/- %0.2f)" % (-scores.mean(), scores.std() * 2))

/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


Median absolute error: 30.80 (+/- 0.26)


In [50]:
y_pred = regr.predict(X_test)
error = median_absolute_error(y_test, y_pred)
print('Median absolute error on test set: %.2f' % error)

Median absolute error on test set: 30.86


#### Step 3. Improve baseline model.

- Linear regression with outliers removed

In [14]:
def remove_outliers(train):
    df = df_step1.iloc[train.index]

    summary = df.groupby(['direction','route_interval']).agg({'time_diff_seconds': [np.mean, np.std]})
    summary = summary.reset_index()
    summary.columns = ['direction','route_interval','mean','std']
    summary['min'] = summary['mean'] - 1.5 * summary['std'] 
    summary['max'] = summary['mean'] + 1.5 * summary['std'] 

    df = df.merge(summary, left_on = ['direction','route_interval'], 
             right_on = ['direction','route_interval'], how = 'left')
    df['is_outlier'] = np.where ((df['time_diff_seconds'] > df['min']) & \
                (df['time_diff_seconds'] < df['max']), 0, 1)

    not_outliers = df[df['is_outlier']==0].index
    outliers_num = len(df[df['is_outlier']==1])
    print('Number of outliers removed: ' + str(outliers_num))
    
    train = train.iloc[not_outliers]
    return train  

In [15]:
train_no_ouliters = remove_outliers(train)

Number of outliers removed: 7686


In [16]:
X_train = train_no_ouliters[train_no_ouliters.columns[1:]]
y_train = train_no_ouliters[train_no_ouliters.columns[0]]

regr_improved = LinearRegression()
regr_improved.fit(X_train, y_train)
scores = cross_val_score(regr_improved, X_train, y_train, cv=5, scoring = 'neg_median_absolute_error')
print("Median absolute error: %0.2f (+/- %0.2f)" % (-scores.mean(), scores.std() * 2))

Median absolute error: 27.44 (+/- 0.21)


In [53]:
y_pred = regr_improved.predict(X_test)
error = median_absolute_error(y_test, y_pred)
print('Median absolute error on test set: %.2f' % error)

Median absolute error on test set: 29.28


- Gradient Boosting Regressor: Initial Tunning

In [17]:
clf = GradientBoostingRegressor(n_estimators = 300)

param_grid = {
              'learning_rate': [0.1, 0.05],
              'max_depth': [4, 6, 8],
              'min_samples_leaf': [5, 9, 11]
             }

In [18]:
grid_search = GridSearchCV(clf, param_grid, scoring = 'neg_median_absolute_error',
             n_jobs=2, cv=3, verbose=2 )

grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] learning_rate=0.1, max_depth=4, min_samples_leaf=5 ..............
[CV] learning_rate=0.1, max_depth=4, min_samples_leaf=5 ..............
[CV]  learning_rate=0.1, max_depth=4, min_samples_leaf=5, total=  34.1s
[CV] learning_rate=0.1, max_depth=4, min_samples_leaf=5 ..............
[CV]  learning_rate=0.1, max_depth=4, min_samples_leaf=5, total=  34.4s
[CV] learning_rate=0.1, max_depth=4, min_samples_leaf=9 ..............
[CV]  learning_rate=0.1, max_depth=4, min_samples_leaf=5, total=  32.6s
[CV] learning_rate=0.1, max_depth=4, min_samples_leaf=9 ..............
[CV]  learning_rate=0.1, max_depth=4, min_samples_leaf=9, total=  32.6s
[CV] learning_rate=0.1, max_depth=4, min_samples_leaf=9 ..............
[CV]  learning_rate=0.1, max_depth=4, min_samples_leaf=9, total=  29.4s
[CV] learning_rate=0.1, max_depth=4, min_samples_leaf=11 .............
[CV]  learning_rate=0.1, max_depth=4, min_samples_leaf=9, total=  29.8s
[CV] learn

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 19.5min


[CV]  learning_rate=0.05, max_depth=6, min_samples_leaf=5, total= 1.0min
[CV] learning_rate=0.05, max_depth=6, min_samples_leaf=9 .............
[CV]  learning_rate=0.05, max_depth=6, min_samples_leaf=5, total= 1.1min
[CV] learning_rate=0.05, max_depth=6, min_samples_leaf=9 .............
[CV]  learning_rate=0.05, max_depth=6, min_samples_leaf=9, total= 1.0min
[CV] learning_rate=0.05, max_depth=6, min_samples_leaf=9 .............
[CV]  learning_rate=0.05, max_depth=6, min_samples_leaf=9, total= 1.1min
[CV] learning_rate=0.05, max_depth=6, min_samples_leaf=11 ............
[CV]  learning_rate=0.05, max_depth=6, min_samples_leaf=9, total= 1.1min
[CV] learning_rate=0.05, max_depth=6, min_samples_leaf=11 ............
[CV]  learning_rate=0.05, max_depth=6, min_samples_leaf=11, total= 1.1min
[CV] learning_rate=0.05, max_depth=6, min_samples_leaf=11 ............
[CV]  learning_rate=0.05, max_depth=6, min_samples_leaf=11, total= 1.1min
[CV] learning_rate=0.05, max_depth=8, min_samples_leaf=5 ....

[Parallel(n_jobs=2)]: Done  54 out of  54 | elapsed: 32.3min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=300, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'learning_rate': [0.1, 0.05], 'max_depth': [4, 6, 8], 'min_samples_leaf': [5, 9, 11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_median_absolute_error', verbose=2)

In [19]:
print('Best Median absolute error: %0.2f' % (-grid_search.best_score_))
print('Best Parameters: ' + str(grid_search.best_params_))

Best Median absolute error: 19.93
Best Parameters: {'learning_rate': 0.1, 'max_depth': 6, 'min_samples_leaf': 11}


- Gradient Boosting Regressor: Tuning min_sample_leaf

In [21]:
clf2 = GradientBoostingRegressor(n_estimators = 300, 
                                 learning_rate = 0.1,
                                 max_depth = 6)

param_grid2 = {'min_samples_leaf': [11, 14, 17, 20]}

In [22]:
grid_search2 = GridSearchCV(clf2, param_grid2, scoring = 'neg_median_absolute_error',
             n_jobs=2, cv=3, verbose=2 )

grid_search2.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] min_samples_leaf=11 .............................................
[CV] min_samples_leaf=11 .............................................
[CV] .............................. min_samples_leaf=11, total= 1.0min
[CV] min_samples_leaf=11 .............................................
[CV] .............................. min_samples_leaf=11, total= 1.0min
[CV] min_samples_leaf=14 .............................................
[CV] .............................. min_samples_leaf=11, total= 1.0min
[CV] min_samples_leaf=14 .............................................
[CV] .............................. min_samples_leaf=14, total= 1.0min
[CV] min_samples_leaf=14 .............................................
[CV] .............................. min_samples_leaf=14, total= 1.0min
[CV] min_samples_leaf=17 .............................................
[CV] .............................. min_samples_leaf=14, total= 1.0min
[CV] min_samples_

[Parallel(n_jobs=2)]: Done  12 out of  12 | elapsed:  6.3min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=6, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=11,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=300, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'min_samples_leaf': [11, 14, 17, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_median_absolute_error', verbose=2)

In [23]:
print('Best Median absolute error: %0.2f' % (-grid_search2.best_score_))
print('Best Parameters: ' + str(grid_search2.best_params_))

Best Median absolute error: 19.93
Best Parameters: {'min_samples_leaf': 11}


- Gradient Boosting Regressor: Final tuning

In [25]:
clf3 = GradientBoostingRegressor(n_estimators = 1000, 
                                 learning_rate = 0.1,
                                 max_depth = 6,
                                 min_samples_leaf = 11)

param_grid3 = {'learning_rate': [0.1, 0.5, 0.02, 0.01]}

In [26]:
grid_search3 = GridSearchCV(clf3, param_grid3, scoring = 'neg_median_absolute_error',
             n_jobs=2, cv=3, verbose=2 )

grid_search3.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] learning_rate=0.1 ...............................................
[CV] learning_rate=0.1 ...............................................
[CV] ................................ learning_rate=0.1, total= 3.6min
[CV] learning_rate=0.1 ...............................................
[CV] ................................ learning_rate=0.1, total= 3.6min
[CV] learning_rate=0.5 ...............................................
[CV] ................................ learning_rate=0.1, total= 3.4min
[CV] learning_rate=0.5 ...............................................
[CV] ................................ learning_rate=0.5, total= 3.7min
[CV] learning_rate=0.5 ...............................................
[CV] ................................ learning_rate=0.5, total= 3.9min
[CV] learning_rate=0.02 ..............................................
[CV] ................................ learning_rate=0.5, total= 4.0min
[CV] learning_rat

[Parallel(n_jobs=2)]: Done  12 out of  12 | elapsed: 22.5min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=6, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=11,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'learning_rate': [0.1, 0.5, 0.02, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_median_absolute_error', verbose=2)

In [27]:
print('Best Median absolute error: %0.2f' % (-grid_search3.best_score_))
print('Best Parameters: ' + str(grid_search3.best_params_))

Best Median absolute error: 20.08
Best Parameters: {'learning_rate': 0.1}


Train and save final model

In [28]:
clf_final = grid_search3.best_estimator_
clf_final.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=6, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=11,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [30]:
model = clf_final
joblib.dump(model, 'model.pkl') 

['model.pkl']

#### Step 4. Model Evaluation

In [42]:
y_pred = clf_final.predict(X_test)
error = median_absolute_error(y_test, y_pred)
print('Median absolute error on test set: %.2f' % error)

Median absolute error on test set: 21.77


In [39]:
df_evaluation = df_step0.iloc[X_test.index]
df_evaluation = df_evaluation[['direction','route_interval', 'time_diff_seconds']]
df_evaluation['predictions'] = y_pred
df_evaluation['error'] = np.abs(df_evaluation['predictions'] - df_evaluation['time_diff_seconds'])

In [47]:
error_summary = df_evaluation.groupby(['direction','route_interval']).agg({'error' : ['count', np.median]})
error_summary = error_summary.reset_index()
error_summary.columns = ['direction','route_interval','n','median_error']
error_summary['error_type'] = np.where(error_summary['median_error'] > error*1.25, 'high',
                              np.where(error_summary['median_error'] < error*0.75, 'low', 'avg'))

In [57]:
error_summary[error_summary['error_type'] == 'high'].sort_values(by = 'median_error', ascending = False)

,direction,route_interval,n,median_error,error_type
25,0,#26_#27,807,108.142714,high
53,1,#27_#28,924,88.727332,high
44,1,#18_#19,932,43.556659,high
1,0,#02_#03,657,33.289124,high
11,0,#12_#13,745,31.618724,high
9,0,#10_#11,748,30.619026,high
14,0,#15_#16,614,30.306155,high
39,1,#13_#14,452,29.956692,high
43,1,#17_#18,940,29.701877,high
26,1,#00_#01,1035,28.627297,high


In [58]:
error_summary[error_summary['error_type'] == 'low'].sort_values(by = 'median_error', ascending = False)

,direction,route_interval,n,median_error,error_type
20,0,#21_#22,418,16.309283,low
27,1,#01_#02,974,16.048940,low
36,1,#10_#11,634,15.712757,low
49,1,#23_#24,688,15.638464,low
38,1,#12_#13,417,15.081569,low
18,0,#19_#20,543,14.772386,low
19,0,#20_#21,571,13.429593,low
24,0,#25_#26,749,13.368453,low
22,0,#23_#24,657,13.253535,low
3,0,#04_#05,566,12.204376,low
